IA Agentic sobre mi perfil de LinkedIn

In [1]:
# LIBRERIAS

from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr

In [2]:
# CARGAR VARIABLES DE ENTORNO Y ENDPOINT DE OPEN AI

load_dotenv(override=True)
openai = OpenAI()

In [3]:
# CARGAR VARIABLES DE ENTORNO PARA LAS NOTIFIACIONES PUSH
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [4]:
# DEFINIR FUNCIÓN PUSHOVER PARA MANDAR NOTIFICACIONES
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [5]:
# DEFINIR FUNCION PARA RECODIGA DE DATOS DE USUARIOS QUE QUIERAN CONTACTAR CONTIGO
def record_user_details(email, name="Nombre no proporcionado", notes="not provided"):
    push(f"Registrando interés de {name} con email {email} y notas {notes}")
    return {"recorded": "ok"}

In [6]:
# DEFINIR FUNCION DE REGISTRO DE PREGUNTAS QUE EL MODELO LLM DE OPENAI CON LA INFORMACIÓN SUMINISTRADA NO SEPA RESPONDER
def record_unknown_question(question):
    push(f"Registrando pregunta no respondida: {question}")
    return {"recorded": "ok"}

In [7]:
# HERRAMIENTAS PARA REGISTRO DE USUARIO O PREGUNTAS EN JSON
record_user_details_json = {
    "name": "record_user_details",
    "description": "Utilice esta herramienta para registrar que un usuario está interesado en estar en contacto y proporcionó una dirección de correo electrónico.",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {"type": "string"},
            "name": {"type": "string"},
            "notes": {"type": "string"}
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Siempre use esta herramienta para registrar cualquier pregunta que no se pueda responder",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {"type": "string"}
        },
        "required": ["question"],
        "additionalProperties": False
    }
}


In [8]:
# HERRAMINETAS GUARDADAS COMO FUNCIONES 
tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}]

In [9]:
# Esta es una forma más elegante de evitar el IF statement.

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Herramienta llamada: {tool_name}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [10]:
# DICCIONARIO PARA BUSCAR CUALQUIER FUNCION DE FORMA GLOBAL
globals()["record_unknown_question"]("esta es una pregunta realmente difícil")

Push: Registrando pregunta no respondida: esta es una pregunta realmente difícil


{'recorded': 'ok'}

In [11]:
# EXTRAER INFORMACION DEL PERFIL DE LINKEDIN
reader = PdfReader("C:/Users/fmr10.000/Github/agents/1_foundations/me/FMR_LINKEDIN.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

with open("C:/Users/fmr10.000/Github/agents/1_foundations/me/SUMMARY_FMR.txt", "r", encoding="utf-8") as f:
    summary = f.read()

name = "Fco Martín"

In [12]:
# DEFINIR EL PROMPT DE SISTEMA
system_prompt = f"""Estás actuando como {name}. Estás respondiendo preguntas en el sitio web de {name}, en particular preguntas relacionadas con la carrera, los antecedentes, las habilidades y la experiencia de {name}.
Tu responsabilidad es representar a {name} en las interacciones en el sitio web con la mayor fidelidad posible.
Se te proporciona un resumen de los antecedentes y el perfil de LinkedIn de {name} que puedes usar para responder preguntas.
Sé profesional y atractivo, como si hablaras con un cliente potencial que haya visitado el sitio web.
Si no sabes la respuesta a alguna pregunta, usa la herramienta record_unknown_question para registrar la pregunta que no pudiste responder, incluso si se trata de algo trivial o no relacionado con tu carrera.
Si el usuario participa en una conversación, intenta que se ponga en contacto por correo electrónico; pídele su correo electrónico y regístralo con la herramienta record_user_details."""

system_prompt += f"\n\n## Resumen:\n{summary}\n\n## LinkedIn Perfil:\n{linkedin}\n\n"
system_prompt += f"En este contexto, chatea con el usuario, siempre con el personaje {name}."

In [13]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    done = False
    while not done:

        # Esta es la llamada a la LLM - nota que pasamos el json de las herramientas

        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)

        finish_reason = response.choices[0].finish_reason
        
        # Si la LLM quiere llamar a una herramienta, la llamamos!
         
        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content

In [14]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [17]:
# =========================
# UI GRADIO (BONITA) + CHAT VISIBLE (CORREGIDO)
# =========================
import gradio as gr

CSS = """
.gradio-container { max-width: 1200px !important; margin: 0 auto !important; }
#app { padding-top: 10px; }

.hero{
  padding:18px 18px 14px 18px; border-radius:18px;
  background: linear-gradient(135deg, rgba(59,130,246,.16), rgba(168,85,247,.12));
  border: 1px solid rgba(255,255,255,.10);
}
.hero h1{ margin:0; font-size:26px; letter-spacing:-.2px; }
.hero p{ margin:6px 0 0 0; opacity:.85; }

.card{
  border-radius:18px; padding:14px;
  background: rgba(255,255,255,.04);
  border: 1px solid rgba(255,255,255,.10);
}

#chatbot{
  border-radius:18px; overflow:hidden;
  border: 1px solid rgba(255,255,255,.10);
}
#msg textarea{ border-radius:16px !important; min-height:62px !important; }

button, .gr-button{ border-radius:14px !important; font-weight:600 !important; }
.chips .gr-button{
  border-radius:999px !important; padding:8px 12px !important; margin:4px 6px 0 0 !important;
  border: 1px solid rgba(255,255,255,.10) !important;
  background: rgba(255,255,255,.04) !important;
}
.chips .gr-button:hover{ transform: translateY(-1px); }

.footer{ opacity:.7; font-size:12px; margin-top:10px; }
"""

SUGERENCIAS = [
    "Dame un resumen profesional en 5 bullets.",
    "Cuéntame 2-3 proyectos destacados y tu impacto.",
    "Redáctame un email para contactarte y qué datos necesitas."
]

with gr.Blocks(
    css=CSS,
    theme=gr.themes.Soft(
        primary_hue="blue",
        secondary_hue="violet",
        neutral_hue="slate",
        radius_size="lg",
        text_size="md"
    ),
    title=f"Chat con {name}",
) as demo:

    with gr.Column(elem_id="app"):
        gr.HTML(f"""
        <div class="hero">
          <h1>👋 Habla con {name}</h1>
          <p>Asistente sobre experiencia, habilidades, proyectos y colaboración.</p>
        </div>
        """)

        with gr.Row(equal_height=True):
            with gr.Column(scale=7):
                chatbot = gr.Chatbot(
                    label="",
                    elem_id="chatbot",
                    height=540,
                    show_copy_button=True,
                    bubble_full_width=False,
                    type="messages"
                )

                msg = gr.Textbox(
                    label="Tu mensaje",
                    elem_id="msg",
                    placeholder="Escribe aquí… (ej: “¿En qué proyectos has trabajado con IA?”)",
                    lines=2
                )

                with gr.Row():
                    send_btn = gr.Button("Enviar", variant="primary")
                    clear_btn = gr.Button("Limpiar", variant="secondary")

                with gr.Column(elem_classes="chips"):
                    gr.Markdown("**Sugerencias rápidas**")
                    with gr.Row():
                        chip_buttons = [gr.Button(s, variant="secondary") for s in SUGERENCIAS]

                gr.HTML('<div class="footer">Tip: pide respuestas en formato CV, bullets o storytelling.</div>')

            with gr.Column(scale=3):
                gr.HTML("""
                <div class="card">
                  <h3 style="margin:0 0 8px 0;">📌 Qué puedes preguntar</h3>
                  <ul style="margin:0; padding-left:18px; opacity:.9;">
                    <li>Resumen y propuesta de valor</li>
                    <li>Experiencia por sector / rol</li>
                    <li>Stack técnico y herramientas</li>
                    <li>Proyectos y resultados</li>
                    <li>Cómo colaborar y contacto</li>
                  </ul>
                </div>
                """)
                gr.HTML("""
                <div class="card" style="margin-top:12px;">
                  <h3 style="margin:0 0 8px 0;">✉️ Contacto</h3>
                  <p style="margin:0; opacity:.9;">
                    Si te interesa colaborar, pide el formato de contacto y deja tu email.
                  </p>
                </div>
                """)

        # ✅ Un solo historial: el del chatbot (NO necesitas state)
        def on_user_submit(user_text, history):
            if not user_text or not user_text.strip():
                return "", history
            history = history + [{"role": "user", "content": user_text.strip()}]
            return "", history

        def on_bot_reply(history):
            user_message = history[-1]["content"]
            prior = history[:-1]
            assistant = chat(user_message, prior)
            history = history + [{"role": "assistant", "content": assistant}]
            return history

        # IMPORTANTE: outputs incluyen chatbot para que se vea
        msg.submit(on_user_submit, [msg, chatbot], [msg, chatbot]).then(on_bot_reply, chatbot, chatbot)
        send_btn.click(on_user_submit, [msg, chatbot], [msg, chatbot]).then(on_bot_reply, chatbot, chatbot)

        clear_btn.click(lambda: [], None, chatbot)

        for i, b in enumerate(chip_buttons):
            b.click(lambda idx=i: SUGERENCIAS[idx], None, msg)

demo.launch()


C:\Users\fmr10.000\AppData\Local\Temp\ipykernel_34644\878317235.py:69: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(


* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.
